In [1]:
#--------------------------------------------------------------------------------
# These tokens are needed for user authentication.
# Credentials can be generates via Forsquare:

#--------------------------------------------------------------------------------

client_id = "3R1DHJA1BIUJMTE0TGMAP54JFTKGPMXMJC0YFUCQVOWZK45W"
client_secret = "4IXEO10NH2TL2HNECZU1ZLMD4GJ3RLMMBMWQQ4W13U4CMHSV"
VERSION = '20180604'


venue_categories =	{ 'entertainment': '4bf58dd8d48988d171941735', 'food': '4bf58dd8d48988d10e941735', 'nightlife': '4bf58dd8d48988d1e5931735', 'museum' : '4bf58dd8d48988d181941735'}

In [2]:
from foursquare import *
import sys
sys.path.append(".")
import os
import re


def file_under_dir(directory_name,filename):
    directory_name = os.getcwd() + directory_name
    filepath = os.path.join(directory_name, filename)
    try: 
        os.makedirs(directory_name,0o777)
    except OSError:
        if not os.path.isdir(directory_name):
            raise
    return filepath

def search_venues(categoryId):  #return a list with the ids of the venues
    venues = client.venues.search(params={'categoryId': categoryId, 'll': '37.9724782,23.729001', 'radius':1000,'limit' : 50}) #arts & entertainment
    venues_ids = []
    for venue in venues['venues']:
        venues_ids.append(venue['id'])
    return venues_ids

def unique_venues_ids(categoryId, filename): #check if we have already information for this venue
    #read old ids from csv file
    with open(filename, 'r') as f:
        old_ids = f.read().splitlines()

    new_ids = search_venues(categoryId)
    if not old_ids: #we have not stored any id
        difference = new_ids
    else:
        difference =  list(set(new_ids) - set(old_ids))
    with open(filename, 'a') as f:  #add different ids to the file with explored ids
        for item in difference:
            f.write("%s\n" % item)
    return difference

def get_details_of_venue(venues_ids,category_name):
    for i in venues_ids:
        venue_details  = client.venues(i)
        with open(file_under_dir('/foursquare/' + category_name + '/','foursquare_' + i +'.json'), 'a', encoding='utf8') as file:
                json.dump(venue_details,file,indent=4,sort_keys=True)
    
        
client = Foursquare(client_id, client_secret, redirect_uri='http://fondu.com/oauth/authorize')


for key, value in venue_categories.items():
    ids = unique_venues_ids(value,key + '.csv')
    get_details_of_venue(ids,key)
    


entertainment
food
nightlife
museum
